## Change Log

#### Dataset & Model Overview

- After getting a decent understanding of how a supervised neural network like the one I did in predicting wine quality with a FFNN, I wanted to try working with image recognition and detection.
- I am using this dataset (https://www.cs.toronto.edu/~kriz/cifar.html), this dataset is a subset of the 80 million tiny images dataset.
- This dataset has 60,000 images with 10 classes and 6000 images per class.
- I want this model to be able to correctly classify the class that the image belongs to.
- I am planning to use a Convolutional Neural Network (CNN) for this, I chose a CNN because they are very good at pattern recognition and image detection with classification which is exactly what is needed for this task.

#### Learnings & Findings

- I am extracting the dataset in the way specified here (https://www.cs.toronto.edu/~kriz/cifar.html) alongside extra code to extract and combine them into a singular csv file.
- I am going to try the CNN structure similar to what is found here (https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/), I used Tensorflow for my last model and it worked well so I am going to try and use it again.
- I had to learn how the image classification is structured in tensorflow, my article that I referenced above has some information on it but I used some other sites for research such as (https://www.kaggle.com/code/anandhuh/image-classification-using-cnn-for-beginners).
- Ran into an issue with how my data was being structured after I changed it to a CSV file. When I was trying to fit the model it was throwing an error because my data was structured incorrectly for the CNN. I was struggling to figure out why but I ended up asking ChatGPT for help here. It told me that the issue is that it was being stored as a 1D array where as the CNN model requires it to be a 3D image. So it converted it to a numpy arrray and then reshaped the data so that it would fit the model better.


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

Here I am just importing the necessary libraries and technologies that I may need to build this model.


In [ ]:
import pickle


def unpickle(file):
    with open(file, "rb") as fo:
        batch = pickle.load(fo, encoding="bytes")
    return batch


data_list = []
labels_list = []

# Loop through all batch files
for i in range(1, 6):
    file = f"data_batch_{i}"  # Modify if needed
    batch = unpickle(file)

    data_list.append(batch[b"data"])  # Image data
    labels_list.append(batch[b"labels"])  # Labels

# Convert lists to NumPy arrays
data = np.vstack(data_list)
labels = np.hstack(labels_list)

# Convert to DataFrame
columns = [f"pixel_{i}" for i in range(data.shape[1])]  # Column names for pixels
df = pd.DataFrame(data, columns=columns)
df.insert(0, "label", labels)  # Insert labels as the first column

# Save to CSV
df.to_csv("cifar10_data.csv", index=False)
print("CSV file saved successfully!")

CSV file saved successfully!


ChatGPT generated this for me as I wanted to store this as a CSV file rather than the way they were initially being handled.

What it is doing is, looping through each of the files which has 10,000 images in it and labels of what each image is of so the model can begin to build up an idea of what should be in each class, it then extracts the data and stores it in a pandas dataframe which can then be converted to a singular csv file.

I am now going to turn the test batch into a csv file as well.


In [ ]:
import pickle


def unpickle(file):
    with open(file, "rb") as fo:
        batch = pickle.load(fo, encoding="bytes")
    return batch


data_list = []
labels_list = []

file = "test_batch"  # Modify if needed
batch = unpickle(file)

data_list.append(batch[b"data"])  # Image data
labels_list.append(batch[b"labels"])  # Labels

# Convert lists to NumPy arrays
data = np.vstack(data_list)
labels = np.hstack(labels_list)

# Convert to DataFrame
columns = [f"pixel_{i}" for i in range(data.shape[1])]  # Column names for pixels
df = pd.DataFrame(data, columns=columns)
df.insert(0, "label", labels)  # Insert labels as the first column

# Save to CSV
df.to_csv("cifar10_test_data.csv", index=False)
print("CSV file saved successfully!")

CSV file saved successfully!


In [ ]:
df = pd.read_csv("cifar10_data.csv")
test_df = pd.read_csv("cifar10_test_data.csv")

# Separate features and labels
X = df.drop("label", axis=1).values  # Convert to NumPy array
y = df["label"].values  # Convert labels to NumPy array

# Reshape X to match CNN input (num_samples, 32, 32, 3)
X = X.reshape(-1, 32, 32, 3)

# Do the same for test data
X_test = test_df.drop("label", axis=1).values  # Convert to NumPy array
y_test = test_df["label"].values  # Convert labels to NumPy array

# Reshape X_test to match CNN input
X_test = X_test.reshape(-1, 32, 32, 3)

# Normalize pixel values (important for CNNs)
X = X / 255.0
X_test = X_test / 255.0

X shape: (50000, 32, 32, 3)
y shape: (50000,)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000,)


Here I'm just assigning both datasets and the target variables for both the training and test dataset.

My next step is now to add the layers to the model that I will need for the classification.

##### Update

I had to get ChatGPT to help me retructure my dataframe. Full explanation in the change log.


In [19]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


This is the most popular structure that I have seen online. After doing some more research about what the functions that are being called are doing it is as follows:

A Sequential model is being used as each layer is using the last layers output as its input, this is important for something like image clasification because we arre filtering and condensing the image so it needs to be executed in sequential order.

The Conv2D function is used to add a convolutional layer, what this does is it sets an amount of features which in this case is 32, and these will scan over the image in a 3x3 grid looking for key edges or features that are standing out. Then I have the ReLU activation function which just turns negative values into 0. Finally the input shape is just telling the model what the shape of my image is so you have to specify the height, width and colourscale so 3 is RGB for example.

The MaxPooling2d function is used to downsize the image that we have. So how this works is it will move across in a 2x2 grid and look for the square with the most important features in it and it removes the rest, this is so it is focusing on the key information and not background and unimportant parts of the image.

This process is then repeated a couple more times so that the model can continue to try and correctly identify the correct image.


In [20]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10))

In [21]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
# Train the model
model.fit(X, y, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.3326 - loss: 1.8263 - val_accuracy: 0.4961 - val_loss: 1.3855
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.5050 - loss: 1.3986 - val_accuracy: 0.5323 - val_loss: 1.3028
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5592 - loss: 1.2424 - val_accuracy: 0.5666 - val_loss: 1.2101
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5951 - loss: 1.1482 - val_accuracy: 0.5682 - val_loss: 1.2232
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6191 - loss: 1.0799 - val_accuracy: 0.6031 - val_loss: 1.1188
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6381 - loss: 1.0239 - val_accuracy: 0.5668 - val_loss: 1.2472
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6524 - loss: 0.9778 - val_accuracy: 0.6054 - val_loss: 1.1299
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6686 -